In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from import_lib.func import sphere, weierstrass, ackley, rosenbrock, schwefel, griewank, rastrigin
from import_lib.GA import population_init, factorial_cost, factorial_rank, skill_factor_best_task, polynomial_mutation, sbx_crossover

In [2]:
'''
CEC'17
'''
tasks = [
    sphere(     50,shift= 0,    limited_space= True, lower_bound= -100, upper_bound= 100),   # 0
    sphere(     50,shift= 80,   limited_space= True, lower_bound= -100, upper_bound= 100),  # 80
    sphere(     50,shift= -80,  limited_space= True, lower_bound= -100, upper_bound= 100), # -80
    weierstrass(25,shift= -0.4, limited_space= True, lower_bound= -0.5, upper_bound= 0.5), # -0.4
    rosenbrock( 50,shift= -1,   limited_space= True, lower_bound= -50, upper_bound= 50),# 0
    ackley(     50,shift= 40,   limited_space= True, lower_bound= -50, upper_bound= 50),    # 40
    weierstrass(50,shift= -0.4, limited_space= True, lower_bound= -0.5, upper_bound= 0.5), # -0.4
    schwefel(   50,shift= 0,    limited_space= True, lower_bound= -500, upper_bound= 500), # 420.9687
    griewank(   50,shift= [-80, 80],limited_space= True, lower_bound= -100, upper_bound= 100), # -80, 80
    rastrigin(  50,shift= [-40, 40],limited_space= True, lower_bound= -50, upper_bound= 50),# -40, 40
]

In [3]:
print(tasks[0].func(tasks[0].encode(np.array([0] * 50))))
print(tasks[1].func(tasks[1].encode(np.array([80] * 50))))
print(tasks[2].func(tasks[2].encode(np.array([-80] * 50))))
print(tasks[3].func(tasks[3].encode(np.array([-0.4] * 25))))
print(tasks[4].func(tasks[4].encode(np.array([0] * 50))))
print(tasks[5].func(tasks[5].encode(np.array([40] * 50))))
print(tasks[6].func(tasks[6].encode(np.array([-0.4] * 50))))
print(tasks[7].func(tasks[7].encode(np.array([420.9687] * 50))))
print(tasks[8].func(tasks[8].encode(np.append(np.array([-80] * 25), np.array([80] * 25)))))
print(tasks[9].func(tasks[9].encode(np.append(np.array([-40] * 25), np.array([40] * 25)))))


0.0
0.0
0.0
0.0
0.0
4.440892098500626e-16
0.0
0.0006363918728311546
0.0
0.0


In [4]:
print(tasks[0].encode(np.array([0] * 50))[[0, -1]])
print(tasks[1].encode(np.array([80] * 50))[[0, -1]])
print(tasks[2].encode(np.array([-80] * 50))[[0, -1]])
print(tasks[3].encode(np.array([-0.4] * 25))[[0, -1]])
print(tasks[4].encode(np.array([0] * 50))[[0, -1]])
print(tasks[5].encode(np.array([40] * 50))[[0, -1]])
print(tasks[6].encode(np.array([-0.4] * 50))[[0, -1]])
print(tasks[7].encode(np.array([420.9687] * 50))[[0, -1]])
print(tasks[8].encode(np.append(np.array([-80] * 25), np.array([80] * 25)))[[0, -1]])
print(tasks[9].encode(np.append(np.array([-40] * 25), np.array([40] * 25)))[[0, -1]])

[0.5 0.5]
[0.1 0.1]
[0.9 0.9]
[0.9 0.9]
[0.52 0.52]
[0.1 0.1]
[0.9 0.9]
[0.9209687 0.9209687]
[0.9 0.1]
[0.9 0.1]


In [5]:
def Success_History_Memory_Update(M, next_update_position_M, S, delta, K):

    for i in range(K):
        for j in range(K):
            if len(S[i][j]) != 0:
                M[i, j][next_update_position_M[i, j]] =\
                    np.sum(np.array(delta[i][j]) * np.array(S[i][j])**2)/np.sum(np.array(delta[i][j]) * (np.array(S[i][j]))+ 1e-10)
                
                next_update_position_M[i, j] = (next_update_position_M[i, j] + 1) % M.shape[2]

    return M, next_update_position_M

In [6]:
def cus_MFEA(num_epochs, num_inds_each_task, nc = 15, nm = 15, rm = 0.02, mutation_all_gen:bool = False, tasks = [],
            sigmoid = 0.1, H = 30,
            one_line = False, num_epochs_printed = 20 ):
    #save history of factorial cost
    history_cost = np.empty((0, len(tasks)), np.float)
    
    max_d = 0
    for t in tasks:
        if max_d < t.d:
            max_d = t.d

    # initial population and skill_factor_arr
    population, skill_factor_arr = population_init(
        N = num_inds_each_task, 
        num_tasks = len(tasks), 
        d = max_d,
    )
    
    # get skill_factor for pop
    skill_factor_arr = skill_factor_best_task(population, tasks)
    population_factorial_cost = factorial_cost(population, skill_factor_arr, tasks)


    history_cost = np.append(history_cost, [[np.min(population_factorial_cost[np.where(skill_factor_arr == idx)[0]]) for idx in range (len(tasks))]], axis = 0)

    #Inittialize success history mem M
    M = np.ones((len(tasks), len(tasks), H)) / 2
    next_update_position_M = np.zeros((len(tasks), len(tasks)), dtype= np.int)

    # mean and sd of rmp each generations
    # len(task) * len(tasks) * generations * 2
    saved_rmp = np.array([[[[0.5, 0]]]* len(tasks)] * len(tasks)).tolist()
    
    for epoch in range(num_epochs):

        S = np.empty((len(tasks), len(tasks), 0)).tolist()
        delta = np.empty((len(tasks), len(tasks), 0)).tolist()

        #initial offspring for epoch
        offspring = np.empty((0, max_d))
        offspring_skill_factor = np.empty((0, ), np.int)
        offspring_factorial_cost = np.empty((0, ), np.int)

        # his rmp of this generation
        rmp_this_gen = np.empty((len(tasks), len(tasks), 0)).tolist()

        while len(offspring) < len(population):
            skf_pa, skf_pb = np.random.choice(np.arange(len(tasks)), size = 2, replace= False)          

            mu = np.random.choice(M[skf_pa, skf_pb])
            rmp = np.random.normal(mu, sigmoid)
            if rmp < 0: rmp = 0
            if rmp > 1: rmp = 1

            # save generation's rmp
            rmp_this_gen[skf_pa][skf_pb].append(rmp)

            # Inter-task crossover
            if np.random.uniform() <= rmp*(len(tasks) - 1)/len(tasks):
                idx_pa = np.random.choice(np.where(skill_factor_arr == skf_pa)[0])
                idx_pb = np.random.choice(np.where(skill_factor_arr == skf_pb)[0])
                pa, pb = population[idx_pa], population[idx_pb]

                oa, ob = sbx_crossover(pa, pb, nc)
                skf_oa, skf_ob = skf_pa, skf_pa
            # Intra-task
            else:
                idx_pa = np.random.choice(np.where(skill_factor_arr == skf_pa)[0])
                idx_pb = np.random.choice(np.where(skill_factor_arr == skf_pa)[0])
                pa, pb = population[idx_pa], population[idx_pb]

                oa, ob = sbx_crossover(pa, pb)
                skf_oa, skf_ob = skf_pa, skf_pa

            # mutation
            oa = polynomial_mutation(oa,nm, rm, mutation_all_gen)
            ob = polynomial_mutation(ob,nm, rm, mutation_all_gen)   
            fcost_oa, fcost_ob = tasks[skf_oa].func(oa), tasks[skf_ob].func(ob)

            Delta = max(0, 
                    (population_factorial_cost[idx_pa] - fcost_oa)/population_factorial_cost[idx_pa],
                    (population_factorial_cost[idx_pa] - fcost_ob)/population_factorial_cost[idx_pa]
                )

            if Delta > 0:
                S[skf_pa][skf_pb].append(rmp)
                delta[skf_pa][skf_pb].append(Delta)    

            offspring = np.append(offspring, [oa, ob], axis = 0)
            offspring_skill_factor = np.append(offspring_skill_factor, [skf_oa, skf_ob])
            offspring_factorial_cost = np.append(offspring_factorial_cost, [fcost_oa, fcost_ob])

        M, next_update_position_M = Success_History_Memory_Update(M, next_update_position_M, S, delta, K = len(tasks))

        # update pop
        population = np.append(population, offspring, axis = 0)
        skill_factor_arr = np.append(skill_factor_arr, offspring_skill_factor, axis = 0)
        population_factorial_cost = np.append(population_factorial_cost, offspring_factorial_cost, axis = 0)

        # selection
        scalar_fitness = 1/factorial_rank(population_factorial_cost, skill_factor_arr, len(tasks))
        num_inds = np.int(num_inds_each_task)* len(tasks)
        idx_selected_inds = np.argsort(-scalar_fitness)[:num_inds] 
        population, skill_factor_arr = population[idx_selected_inds], skill_factor_arr[idx_selected_inds]
        population_factorial_cost = population_factorial_cost[idx_selected_inds]

        #save history
        history_cost = np.append(history_cost, [[np.min(population_factorial_cost[np.where(skill_factor_arr == idx)[0]]) for idx in range (len(tasks))]], axis = 0)
        
        #save mu and sigmoid rmp
        for i in range (len(tasks)):
            for j in range (len(tasks)):
                if len(rmp_this_gen[i][j]) != 0:
                    mu = np.mean(rmp_this_gen[i][j])
                    sd = np.std(rmp_this_gen[i][j])
                    saved_rmp[i][j].append([mu, sd])
        
        #print
        if (epoch + 1) % (num_epochs // num_epochs_printed) == 0:
            if one_line == True:
                sys.stdout.write('\r')
            sys.stdout.write('Epoch [{}/{}], [%-20s] %3d%% ,func_val: {}'
                .format(epoch + 1, num_epochs,[np.min(population_factorial_cost[np.where(skill_factor_arr == idx)[0]]) for idx in range (len(tasks))])
                % ('=' * ((epoch + 1) // (num_epochs // 20)) + '>' , (epoch + 1) * 100 // num_epochs)
                )
            if one_line == False:
                print("\n")
            sys.stdout.flush()

    print('END!')
    #find solve
    sol_idx = [np.argmin(population_factorial_cost[np.where(skill_factor_arr == idx)]) for idx in range (len(tasks))]
    sol = [task.decode(population[np.where(skill_factor_arr == idx)][sol_idx[idx]]) for idx, task in enumerate(tasks)]

    return sol, history_cost, np.array(saved_rmp)


In [7]:
cus_sol, cus_his, cus_rmp_his = cus_MFEA(num_epochs= 1000, num_inds_each_task= 100, tasks = tasks,
                        nc = 2, nm = 5, rm = 0.02, mutation_all_gen= False,
                        sigmoid= 0.1, H = 30, 
                        one_line=False, num_epochs_printed=10)


Epoch [100/1000], [==>                 ]  10% ,func_val: [2235.989123517251, 14886.698670080013, 24960.44134448116, 16.239149918343024, 1405492.9520953062, 18.681062647968435, 39.02109874675754, 4823.259654783766, 13.335253150472749, 11596.742989622451]

Epoch [200/1000], [====>               ]  20% ,func_val: [716.3143263502008, 92.08146519546337, 2752.1893693639768, 7.780630839047859, 125222.80438934731, 4.258238940971761, 25.83486673867833, 3000.319498388162, 2.034795977264804, 1397.2466973657415]

Epoch [300/1000], [======>             ]  30% ,func_val: [260.95426272342087, 43.20818153301036, 674.3681690925821, 3.575606297362448, 26664.55009532986, 3.1778374114577486, 18.59468369136512, 2631.3933056696187, 1.1835079421263206, 622.2052639848324]



KeyboardInterrupt: 

In [ ]:
np.save(file = "./RESULTS/custom_his_CEC17.npy", arr = cus_his)
np.save(file = "./RESULTS/custon_rmp_his_CEC17.npy", arr = cus_rmp_his )

In [ ]:
fig = plt.figure(figsize = (30, 10), dpi= 200)
fig.suptitle("Compare\n", size = 15)
fig.set_facecolor("white")

label_legend = ["custom MFEA"]

for i in range (len(tasks)):
    for idx, sub_his in enumerate([cus_his]): 
        plt.subplot(2, int(len(tasks) / 2), i + 1)

        plt.plot(np.arange(0, 1000, 100), sub_his[np.arange(0, 1000, 100), i], label= label_legend[idx])
        plt.scatter(np.arange(0, 1000, 100), sub_his[np.arange(0, 1000, 100), i])
        plt.legend()

    plt.title(tasks[i].name)
    plt.xlabel("Epoch")
    plt.ylabel("F(x)")
    plt.yscale("log")

In [ ]:
mfea_his = np.load(file = "./RESULTS/mfea_base_his_CEC17.npy")

sa_his = np.load(file = "./RESULTS/sa_his_CEC17.npy")
sa_rmp_his  = np.load(file = "./RESULTS/sa_rmp_his_CEC17.npy", allow_pickle= True)

lsa_his = np.load(file = "./RESULTS/lsa_his_CEC17.npy")
lsa_rmp_his  = np.load(file = "./RESULTS/lsa_rmp_his_CEC17.npy", allow_pickle= True)

cus_his  = np.load(file = "./RESULTS/custom_his_CEC17.npy")
cus_rmp_his  = np.load(file = "./RESULTS/custon_rmp_his_CEC17.npy", allow_pickle= True)

In [ ]:
fig = plt.figure(figsize = (30, 10), dpi= 200)
fig.suptitle("Compare\n", size = 15)
fig.set_facecolor("white")

label_legend = ["MFEA", "SA", "LSA MFEA", "custom MFEA"]

for i in range (len(tasks)):
    for idx, sub_his in enumerate([mfea_his, sa_his, lsa_his, cus_his]): 
        plt.subplot(2, int(len(tasks) / 2), i + 1)

        plt.plot(np.arange(0, 1000, 100), sub_his[np.arange(0, 1000, 100), i], label= label_legend[idx])
        plt.scatter(np.arange(0, 1000, 100), sub_his[np.arange(0, 1000, 100), i])
        plt.legend()

    plt.title(tasks[i].name)
    plt.xlabel("Epoch")
    plt.ylabel("F(x)")
    plt.yscale("log")

In [ ]:
fig = plt.figure(figsize = (30, 20), dpi= 200)
fig.suptitle("cusMFEA RMP Val\n", size = 15)
fig.set_facecolor("white")

for i in range (len(tasks)):
    for j in range (len(tasks)):
        r, c = min(i, j), max(i, j)
        if i != j: 
            plt.subplot(int(np.ceil(len(tasks) / 3)), 3, i + 1)
            plt.plot(np.arange(len(cus_rmp_his[r][c])), np.array(cus_rmp_his[r][c])[:, 0], label= 'task: ' +str(j + 1))
            plt.legend()
        else:
            plt.subplot(int(np.ceil(len(tasks) / 3)), 3, i + 1)
            plt.plot(np.arange(1000), np.ones_like(np.arange(1000)), label= 'task: ' +str(j + 1))
            plt.legend()




    plt.title('task ' + str( i + 1))
    plt.xlabel("Epoch")
    plt.ylabel("M_rmp")
    plt.ylim(bottom = -0.1, top = 1.1)
